# Setup

Packages we (might) use.

In [1]:
using DrWatson
@quickactivate "SMLP2020"

using MixedModels
using CSV, DataFrames, DataFramesMeta, RCall 
using Statistics: mean, var

# Specification

## Response, covariates, and factors

Linear mixed models (LMMs), like many other types of statistical models, describe a relationship between a *response* variable and *covariates* that have been measured or observed along with the response. The statistical model assumes that the residuals of the fitted response (i.e., not the responses) are normally -- also identically and independently -- distributed. This is the *first assumption* of normality in the LMM. It is standard practice that model residuals are inspected and, if serious skew is indicated, that the response is Box-Cox transformed (unless not justified for theoretical reasons) to fulfill this model assumption. 

In the following we distinguish between *categorical covariates* and *numerical covariates*. Categorical covariates are  *factors*. The important characteristic of a factor is that, for each observed value of the response, the factor takes on the value of one of a set of discrete levels.  The levels can be unordered (nominal) or ordered (ordinal). We use the term *covariate* when we refer to *numerical covariates*, that is to continuous measures with some distribution. In principle, statistical models are not constrained by the distribution of observations across levels of factors and covariates, but the distribution may lead to problems of model identification and it does implications for statistical power. 

Statistical power, especially for the detection of interactions, is best when observations are uniformly distributed across levels of factors or uniform across the values of covariates. In experimental designs, uniform distributions may be achieved by balanced assignment of subjects (or other carriers of responses) to the levels of factors or combinations of factor levels. In observational contexts, we achieve uniform distributions by stratification (e..g., on age, gender, or IQ scores). Statistical power is worse for skewed than normal distributions (I think ...). Therefore, although it is *not* required to meet an assumption of the statistical model, it may be useful to consider Box-Cox transformations of covariates.

## Nested and crossed random (grouping) factors

In LMMs the levels of at least one of the factors represents *units* in the data set that are assumed to be sampled, ideally randomly, from a population that is normally distributed with respect to the response. *This is the second assumption of normal distribution in LMMs.*  In psychology and linguistics the observational units are often the subjects or items (e..g., texts, sentences, words, pictures) in the study. We may use numbers, such as subject identifiers, to designate the particular levels that we observed; we recommend to prepend these numbers with "S" or "I" to avoid confusion with numeric variables.

Random sampling is the basis of generalization from the sample to the population. The core statistics we will estimate in this context are variances and correlations of grand means and (quasi-)experimental effects. These terms will be explained below. What we want to stress here is that the estimation of (co-)variances / correlations requires a larger number of units (levels) than the estimation of means. Therefore, from a practical perspective, it is important that random factors are represented with many units.

When there is more than one random factor, we must be clear about their relation. The two prototypical cases are that the factors are *nested* or *crossed*.  In multilevel models, a special case of mixed models, the levels of the random factors are strictly nested. For example, at a given time, every student attends a specific class in a specific school. Students, classes, and schools could be three random factors. As soon as we look at this scenario across several school years, the nesting quickly falls apart because students may move between classes and between schools. 

In psychology and linguistics, random factors are often crossed, for example, when every subject reads every word in every sentence in a word-by-word self-paced reading experiment (or alternatively: when every word in every sentence elicits a response from every subject). However, in an eye-movement experiment (for example), the perfect crossing on a measure like fixation duration is not attainable because of blinks or skipping of words.

In summary, the typical situation in experimental and observational studies with more than one random factor is _partial crossing_ or _partial nesting_ of levels of the random factors. Linear mixed models handle these situations very well. 

## Experimental and quasi-experimental fixed factors / covariates

*Fixed experimental factor or covariate*. In experiments the units (or levels) of the random factor(s) are assigned to manipulations implemented in their design. The researcher controls the assignment of units of the random factor(s) (e.g., subjects, items) to experimental manipulations. These manipulations are represented as factors with a fixed and discrete set of levels (e.g., training vs. control group) or as covariates associated with continuous numeric values (e.g., presentation times). 

*Fixed quasi-experimental factor or covariate*. In observational studies (which can also be experiments) the units (or levels) of random factors may "bring along" characteristics that represent the levels of quasi-experimental factors or covariates beyond the control of the researcher. Whether a a subject is female, male, or diverse or whether a word is a noun, a verb, or an adjective are examples of quasi-experimental factors of gender or word type, respectively. Subject-related covariates are body height, body mass, and IQ scores; word-related covariates are their lengths, frequency, and cloze predictability. 

## Between-unit and within-unit factors / covariates

The distinction between between-unit and within-unit factors is always relative to a random (grouping) factor of an experimental design. A between-unit factor / covariate is a factor for which every unit of the random factor is assigned to or characterized by only one level of the factor. A within-unit factor is a factor for which units of the random factor appear at every level of the factor. 

For the typical random factor, say *Subject*, there is little ambiguity because we are used to the between-within distinction from ANOVAs, more specifically the F1-ANOVA. In psycholinguistics, there is the tradition to test effects also for the second random factor *Item* in an F2-ANOVA. Importantly, for a given fixed factor all four combinations are possible. For example, *Gender* is a fixed quasi-experimental between-subject / within-item factor; word frequency is a fixed quasi-experimental within-subject / between-item covariate; *Pime-target relation* is a fixed experimental  within-subject / within-item factor (assuming that targets are presented both in a primed and in an unprimed situation); and when a training manipulation is defined by the items used in the training, then in a training-control group design, the fixed factor *Group* is a fixed experimental between-subject / between-item factor.    

These distinctions are critical for setting up LMMs because variance components for (quasi-)experimental effects can only be specified for within-unit effects. Note also that loss of data (within limits), counterbalancing or blocking of items are irrelevant for these definitions. 

## Factor-based contrasts and covariate-based trends

The simplest fixed factor has two levels and the model estimates the difference between them. When we move to factors with *k*  levels, we must decide on how we *spend* the *k-1* degrees of freedom, that is we must specify a set of contrasts. (If we don't do it, the program chooses dummy contrasts for us.)

The simplest specification of a covariate is to include its linear trend, that is its slope. The slope (like a contrast) represents a difference score, that is the change in response to a one-unit change on the covariate. For covariates we must decide on the order of the trend we want to model.

## Contrast- and trend-based fixed-effect model parameters 

Fixed factors and covariates are expected to have effects on the response. Fixed-effect model parameters estimate the hypothesized main and interaction effects of the study. The estimates of factors are based on contrasts; the estimates of covariates are based on trends. Conceptually, they correspond to unstandardized regression coefficients in multiple regression. 

The intercept is a special regression coefficient; it estimates the value of the dependent variable when all fixed effects associated with factors and trends associated with covariates are zero. In experimental designs with higher order interactions there is an advantage of specifying the LMM in such a way that the intercept estimates the grand mean (GM). This happens if (a) contrasts for factors are chosen such that the intercept estimates the GM (positive: Sum, SeqDifference, or Helmert contrasts; negative: Dummy contrasts), (b) orthogonal polynomial trends are used (Helmert, anova-based), and (c) covariates are centered on their mean before inclusion in the model. As always, there may be good theoretical reasons to depart from the default recommendation. 

The specification of contrasts / trends does not depend on the status of the fixed factor / covariate. It does not matter whether a factor varies between or within the units of a random factor or whether it is an experimental or quasi-experimental factor. Contrasts are *not* specified for random (grouping) factors.

## Variance components (VCs) and correlation parameters (CPs)

Variance components (VCs) and correlation parameters (CPs) are within-group model parameters; they correspond to (some of the) *within-unit* (quasi-)experimental fixed-effect model parameters. Thus, we may be able to estimate a subject-related VC for word frequency. If we included a linear trend for word frequency, the VC estimates the subject-related variance in these slopes. We cannot estimate an item-related VC for the word-frequency slopes because there is only one frequency associated with words. Analogously, we may able to estimate an item-related VC for the effect of `Group (training vs. control)`, but we cannot estimate a subject-related VC for this effect. 

The within-between characteristics of fixed factors and covariates relative to the random factor(s) are features of the design of the experiment or observational study. They fundamentally constrain the specification of the LMM. That's why it is of upmost importance to be absolutely clear about their status.  

## Conditional modes of random effects

In this outline of the dimensions underlying the specification of an LMM, we have said nothing so far about the conditional modes of random effects (i.e., the results shown in caterpillar and shrinkage plots). They are not needed for model specification or model selection.  

The VC is the prior variance of the random effects, whereas `var(ranef(model))` is the variance of the posterior means/modes of the random effects. See Kliegl et al. (2010, VisualCognition); [Rizopoulos (2019, stackexchange](https://stats.stackexchange.com/questions/392283/interpreting-blups-or-varcorr-estimates-in-mixed-models/392307#392307).

# `MRK17_Exp1.rds` data 

## Preprocessing

We read the data preprocessed with R and saved as RDS file (see `DataPrep.Rmd` for details).

In [2]:
dat = rcopy(R"readRDS($datadir('MRK17_Exp1.rds'))");
describe(dat)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Subj,,S01,,S73,73,,"CategoricalValue{String,UInt32}"
2,Item,,CAKE,,THIEF,240,,"CategoricalValue{String,UInt32}"
3,trial,239.958,2,239.0,480,,,Int64
4,F,,LF,,HF,2,,"CategoricalValue{String,UInt32}"
5,P,,unr,,rel,2,,"CategoricalValue{String,UInt32}"
6,Q,,deg,,clr,2,,"CategoricalValue{String,UInt32}"
7,lQ,,deg,,clr,2,,"CategoricalValue{String,UInt32}"
8,lT,,NW,,WD,2,,"CategoricalValue{String,UInt32}"
9,rt,647.173,301,601.0,2994,,,Int64


The levels of several factors are not in the desired order. We reorder factor levels such that the level with the expected slow response is the second level. This way the fixed effect will be estimated as a positive value.

In [3]:
dat = @transform(dat,
                 F = levels!(:F, ["HF", "LF"]),
                 P = levels!(:P, ["rel", "unr"]),
                 Q = levels!(:Q, ["clr", "deg"]),
                lQ = levels!(:lQ, ["clr", "deg"]),
                lT = levels!(:lT, ["WD", "NW"]));

cellmeans = by(dat, [:F, :P, :Q, :lQ, :lT], 
            meanRT = :rt => mean, sdRT = :rt => std, n = :rt => length,
            semean = :rt => x -> std(x)/sqrt(length(x)))

,F,P,Q,lQ,lT,meanRT,sdRT,n,semean
,Cat…,Cat…,Cat…,Cat…,Cat…,Float64,Float64,Int64,Float64
1,HF,rel,clr,clr,WD,613.094,192.13,499,8.60094
2,HF,rel,clr,clr,NW,635.319,205.19,546,8.78134
3,HF,rel,clr,deg,WD,620.569,173.764,504,7.74007
4,HF,rel,clr,deg,NW,615.647,176.826,527,7.70268
5,HF,rel,deg,clr,WD,667.685,231.145,517,10.1657
6,HF,rel,deg,clr,NW,645.493,179.962,507,7.9924
7,HF,rel,deg,deg,WD,637.793,177.496,546,7.59611
8,HF,rel,deg,deg,NW,659.266,192.637,508,8.54688
9,HF,unr,clr,clr,WD,620.838,192.689,532,8.35414


## Complex LMM

This is *not* the maximal factorial LMM because we do not include interaction terms and associated correlation parameters in the RE structure.

In [4]:
cntrsts = merge(
    Dict(index => EffectsCoding() for index in (:F, :P, :Q, :lQ, :lT)),
    Dict(index => Grouping() for index in (:Subj, :Item)),
);

m1form = @formula (-1000/rt) ~ 1+F*P*Q*lQ*lT +
                              (1+F+P+Q+lQ+lT | Subj) +
                              (1+  P+Q+lQ+lT | Item);
cmplxLMM = fit(MixedModel, m1form, dat, contrasts=cntrsts);

In [5]:
VarCorr(cmplxLMM)

Variance components:
            Column      Variance      Std.Dev.    Corr.
Item     (Intercept)  0.003202035086 0.056586527
         P: unr       0.000129363970 0.011373828 -0.05
         Q: deg       0.000159100613 0.012613509 -0.36 +0.38
         lQ: deg      0.000034951970 0.005912019 -0.37 +0.02 +0.03
         lT: NW       0.000157413709 0.012546462 +0.11 -0.87 -0.01 -0.35
Subj     (Intercept)  0.030558963672 0.174811223
         F: LF        0.000028917819 0.005377529 -0.44
         P: unr       0.000129382006 0.011374621 -0.35 +0.99
         Q: deg       0.000788337211 0.028077343 -0.41 +0.71 +0.70
         lQ: deg      0.000115295913 0.010737593 -0.06 +0.16 +0.16 +0.58
         lT: NW       0.001045670820 0.032336834 -0.26 -0.02 -0.05 +0.37 +0.50
Residual              0.085716454639 0.292773726


In [6]:
cmplxLMM.PCA

(Item = 
Principal components based on correlation matrix
  1.0     ⋅      ⋅      ⋅     ⋅ 
 -0.05   1.0     ⋅      ⋅     ⋅ 
 -0.36   0.38   1.0     ⋅     ⋅ 
 -0.37   0.02   0.03   1.0    ⋅ 
  0.11  -0.87  -0.01  -0.35  1.0
Normalized cumulative variances:
[0.4211, 0.6855, 0.9048, 1.0, 1.0]
Component loadings
 -0.3    0.67  -0.02   0.67  -0.07
  0.6    0.38   0.21  -0.05   0.67
  0.31  -0.34   0.7    0.47  -0.28
  0.31  -0.41  -0.63   0.55   0.2
 -0.6   -0.34   0.27   0.15   0.66, Subj = 
Principal components based on correlation matrix
  1.0     ⋅      ⋅     ⋅     ⋅    ⋅ 
 -0.44   1.0     ⋅     ⋅     ⋅    ⋅ 
 -0.35   0.99   1.0    ⋅     ⋅    ⋅ 
 -0.41   0.71   0.7   1.0    ⋅    ⋅ 
 -0.06   0.16   0.16  0.58  1.0   ⋅ 
 -0.26  -0.02  -0.05  0.37  0.5  1.0
Normalized cumulative variances:
[0.5104, 0.7663, 0.9109, 0.9725, 1.0, 1.0]
Component loadings
 -0.33  -0.02  -0.83  -0.43   0.08   0.08
  0.51   0.34  -0.08  -0.17  -0.27   0.72
  0.5    0.35  -0.18  -0.23  -0.27  -0.69
  0.52  -0.14  

Variance-covariance matrix of random-effect structure suggests overparameterization
for both subject-related and item-related components.

We don't look at fixed effects before model selection.

### VCs and CPs

We can also look separately at item- and subj-related VPs and CPs at a different level.

In [7]:
first(cmplxLMM.λ)

5×5 LinearAlgebra.LowerTriangular{Float64,Array{Float64,2}}:
  0.193277      ⋅            ⋅            ⋅          ⋅ 
 -0.00202368   0.0387958     ⋅            ⋅          ⋅ 
 -0.0155872    0.0154923    0.0370561     ⋅          ⋅ 
 -0.00746083   3.41629e-6  -0.00242281   0.0186072   ⋅ 
  0.00490163  -0.0370277    0.0171984   -0.012066   0.0

VP is zero for last diagonal entry; not supported by data.

In [8]:
last(cmplxLMM.λ)

6×6 LinearAlgebra.LowerTriangular{Float64,Array{Float64,2}}:
  0.597086      ⋅           ⋅          ⋅          ⋅           ⋅ 
 -0.00815559   0.0164576    ⋅          ⋅          ⋅           ⋅ 
 -0.0134575    0.036446    0.0         ⋅          ⋅           ⋅ 
 -0.0392197    0.0570409  -0.0662138  0.0045742   ⋅           ⋅ 
 -0.00202532   0.0056597  -0.0229463  0.0274795  0.00522481   ⋅ 
 -0.028371    -0.0168555  -0.0562006  0.0137443  0.0756871   0.0451023

## Zero-correlation parameter LMM (factors)

In [9]:
m2form = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
                               zerocorr(1+F+P+Q+lQ+lT | Subj) +
                               zerocorr(1  +P+Q+lQ+lT | Item);

zcpLMM = fit(LinearMixedModel, m2form, dat, contrasts=cntrsts);
VarCorr(zcpLMM)

Variance components:
            Column      Variance      Std.Dev.     Corr.
Item     (Intercept)  0.003202763157 0.0565929603
         P: unr       0.000072375973 0.0085074070   .  
         Q: deg       0.000147845307 0.0121591656   .     .  
         lQ: deg      0.000015850793 0.0039813055   .     .     .  
         lT: NW       0.000116383805 0.0107881326   .     .     .     .  
Subj     (Intercept)  0.030610471888 0.1749584862
         F: LF        0.000015560260 0.0039446496   .  
         P: unr       0.000099560396 0.0099779956   .     .  
         Q: deg       0.000773363413 0.0278094123   .     .     .  
         lQ: deg      0.000119128505 0.0109146005   .     .     .     .  
         lT: NW       0.001060759505 0.0325693031   .     .     .     .     .  
Residual              0.085882836483 0.2930577358


In [10]:
m2form_b = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
        (1 |Subj) + (0 +F | Subj) + (0 + P | Subj) + (0 + Q | Subj) + (0 + lQ | Subj) + (0 + lT | Subj) +
        (1 |Item) +               + (0 + P | Item) + (0 + Q | Item) + (0 + lQ | Item) + (0 + lT | Item);

zcpLMM_b = fit(LinearMixedModel, m2form_b, dat, contrasts=cntrsts);
VarCorr(zcpLMM_b)

Variance components:
            Column      Variance      Std.Dev.     Corr.
Item     (Intercept)  0.003202763157 0.0565929603
         P: unr       0.000072375973 0.0085074070   .  
         Q: deg       0.000147845307 0.0121591656   .     .  
         lQ: deg      0.000015850793 0.0039813055   .     .     .  
         lT: NW       0.000116383805 0.0107881326   .     .     .     .  
Subj     (Intercept)  0.030610471888 0.1749584862
         F: LF        0.000015560260 0.0039446496   .  
         P: unr       0.000099560396 0.0099779956   .     .  
         Q: deg       0.000773363413 0.0278094123   .     .     .  
         lQ: deg      0.000119128505 0.0109146005   .     .     .     .  
         lT: NW       0.001060759505 0.0325693031   .     .     .     .     .  
Residual              0.085882836483 0.2930577358


### VCs and CPs

Look ok. It might be a good idea to prune the LMM. 

## Zero-correlation parameter LMM (indicators)

An alternative solution is to extract the indicators of contrasts from the design matrix. Sometimes RE structures are more conveniently specified with indicator variables (i.e., @ level of contrasts) than factors.

In [11]:
mm = Int.(cmplxLMM.X);  

dat = @linq dat |>
       transform(f = mm[:, 2],
                 p = mm[:, 3],
                 q = mm[:, 4],
                lq = mm[:, 5],
                lt = mm[:, 6]);

dat[1:10, 10:14]

,f,p,q,lq,lt
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1
2,-1,1,-1,1,-1
3,-1,-1,-1,1,1
4,-1,-1,1,1,1
5,-1,1,1,1,-1
6,-1,-1,-1,1,-1
7,-1,-1,1,1,1
8,1,1,1,-1,1
9,1,1,-1,1,-1


We take out correlation parameters.

In [12]:
m2form_c = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
 (1 | Subj) + (0+f | Subj) + (0+p | Subj) + (0+q | Subj) + (0+lq | Subj) + (0+lt | Subj) +
 (1 | Item) +                (0+p | Item) + (0+q | Item) + (0+lq | Item) + (0+lt | Item);

zcpLMM_c = fit(LinearMixedModel, m2form_c, dat, contrasts=cntrsts);
VarCorr(zcpLMM_c)

Variance components:
            Column      Variance      Std.Dev.     Corr.
Item     (Intercept)  0.003202763157 0.0565929603
         p            0.000072375973 0.0085074070   .  
         q            0.000147845307 0.0121591656   .     .  
         lq           0.000015850793 0.0039813055   .     .     .  
         lt           0.000116383805 0.0107881326   .     .     .     .  
Subj     (Intercept)  0.030610471888 0.1749584862
         f            0.000015560260 0.0039446496   .  
         p            0.000099560396 0.0099779956   .     .  
         q            0.000773363413 0.0278094123   .     .     .  
         lq           0.000119128505 0.0109146005   .     .     .     .  
         lt           0.001060759505 0.0325693031   .     .     .     .     .  
Residual              0.085882836483 0.2930577358


In [13]:
m2form_d = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
                               zerocorr(1+f+p+q+lq+lt | Subj) +
                               zerocorr(1  +p+q+lq+lt | Item);

zcpLMM_d = fit(LinearMixedModel, m2form_d, dat, contrasts=cntrsts);
VarCorr(zcpLMM)

Variance components:
            Column      Variance      Std.Dev.     Corr.
Item     (Intercept)  0.003202763157 0.0565929603
         P: unr       0.000072375973 0.0085074070   .  
         Q: deg       0.000147845307 0.0121591656   .     .  
         lQ: deg      0.000015850793 0.0039813055   .     .     .  
         lT: NW       0.000116383805 0.0107881326   .     .     .     .  
Subj     (Intercept)  0.030610471888 0.1749584862
         F: LF        0.000015560260 0.0039446496   .  
         P: unr       0.000099560396 0.0099779956   .     .  
         Q: deg       0.000773363413 0.0278094123   .     .     .  
         lQ: deg      0.000119128505 0.0109146005   .     .     .     .  
         lT: NW       0.001060759505 0.0325693031   .     .     .     .     .  
Residual              0.085882836483 0.2930577358


In [14]:
mods = [cmplxLMM, zcpLMM, zcpLMM_b, zcpLMM_c, zcpLMM_d];
gof_summary = DataFrame(dof=dof.(mods), deviance=deviance.(mods),
              AIC = aic.(mods), AICc = aicc.(mods), BIC = bic.(mods))

,dof,deviance,AIC,AICc,BIC
,Int64,Float64,Float64,Float64,Float64
1,69,7147.92,7285.92,7286.51,7817.61
2,44,7188.49,7276.49,7276.73,7615.54
3,44,7188.49,7276.49,7276.73,7615.54
4,44,7188.49,7276.49,7276.73,7615.54
5,44,7188.49,7276.49,7276.73,7615.54


Results are identical for the three versions of `zcpLMM`.  AIC and BIC suggest to go with the `zcpLMM`.

In [15]:
MixedModels.likelihoodratiotest(cmplxLMM, zcpLMM)

Model Formulae
1: :(-1000 / rt) ~ 1 + F + P + Q + lQ + lT + F & P + F & Q + P & Q + F & lQ + P & lQ + Q & lQ + F & lT + P & lT + Q & lT + lQ & lT + F & P & Q + F & P & lQ + F & Q & lQ + P & Q & lQ + F & P & lT + F & Q & lT + P & Q & lT + F & lQ & lT + P & lQ & lT + Q & lQ & lT + F & P & Q & lQ + F & P & Q & lT + F & P & lQ & lT + F & Q & lQ & lT + P & Q & lQ & lT + F & P & Q & lQ & lT + MixedModels.ZeroCorr((1 + F + P + Q + lQ + lT | Subj)) + MixedModels.ZeroCorr((1 + P + Q + lQ + lT | Item))
2: :(-1000 / rt) ~ 1 + F + P + Q + lQ + lT + F & P + F & Q + P & Q + F & lQ + P & lQ + Q & lQ + F & lT + P & lT + Q & lT + lQ & lT + F & P & Q + F & P & lQ + F & Q & lQ + P & Q & lQ + F & P & lT + F & Q & lT + P & Q & lT + F & lQ & lT + P & lQ & lT + Q & lQ & lT + F & P & Q & lQ + F & P & Q & lT + F & P & lQ & lT + F & Q & lQ & lT + P & Q & lQ & lT + F & P & Q & lQ & lT + (1 + F + P + Q + lQ + lT | Subj) + (1 + P + Q + lQ + lT | Item)
──────────────────────────────────────────────────
     model-d

# A zerocorrelation-parameter pecularity of lme4

In [17]:
@rput dat;

### Complex LMM with correlation parameters (factors)

In [18]:
R"""
library(lme4)
contrasts(dat$F) <- -contr.sum(2)
contrasts(dat$P) <- -contr.sum(2)
m1 <- lmer((-1000/rt) ~ 1 + F + P + (1 + F + P | Subj), 
          dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE))
"""

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /Users/reinholdkliegl/.julia/packages/RCall/Qzssx/src/io.jl:160


RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + F + P + (1 + F + P | Subj)
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8012.304  8089.360 -3996.152  7992.304     16399 
Random effects:
 Groups   Name        Std.Dev. Corr       
 Subj     (Intercept) 0.17563             
          F1          0.00558  -0.30      
          P1          0.01200  -0.41  0.99
 Residual             0.30553             
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)           F1           P1  
   -1.63965      0.01831      0.01868  


### Complex LMM with correlation parameters (indicators)

In [19]:
R"""
head(dat)
m1 <- lmer((-1000/rt) ~ 1 + f + p + (1 + f + p | Subj), 
           dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE))
"""

RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + f + p + (1 + f + p | Subj)
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8012.304  8089.360 -3996.152  7992.304     16399 
Random effects:
 Groups   Name        Std.Dev. Corr       
 Subj     (Intercept) 0.17563             
          f           0.00558  -0.30      
          p           0.01200  -0.41  0.99
 Residual             0.30553             
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)            f            p  
   -1.63965      0.01831      0.01868  


### Zerocorr LMM  (indicators)

In [20]:
R"""
m1 <- lmer((-1000/rt) ~ 1+f+p+ (1 | Subj) + (0+f | Subj) +  (0+p | Subj), 
            dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE));
"""

RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + f + p + (1 | Subj) + (0 + f | Subj) + (0 + p |  
    Subj)
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8011.503  8065.442 -3998.752  7997.503     16402 
Random effects:
 Groups   Name        Std.Dev. 
 Subj     (Intercept) 0.1757587
 Subj.1   f           0.0002314
 Subj.2   p           0.0117002
 Residual             0.3055957
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)            f            p  
   -1.63963      0.01831      0.01866  


### Zerocorr LMM - double-bar syntax (indicators) 

In [21]:
R"""
m1 <- lmer((-1000/rt) ~ 1 + f + p + (1 + f + p || Subj), 
           dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE))
"""

RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + f + p + ((1 | Subj) + (0 + f | Subj) + (0 +  
    p | Subj))
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8011.503  8065.442 -3998.752  7997.503     16402 
Random effects:
 Groups   Name        Std.Dev. 
 Subj     (Intercept) 0.1757587
 Subj.1   f           0.0002314
 Subj.2   p           0.0117002
 Residual             0.3055957
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)            f            p  
   -1.63963      0.01831      0.01866  


### Zerocorr LMM - double-bar syntax (factors) - does not return expected results!

In [22]:
R"""
m2 <- lmer((-1000/rt) ~ 1 + F + P + (1 + F + P || Subj), 
            dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE))
"""

RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + F + P + ((1 | Subj) + (0 + F | Subj) + (0 +  
    P | Subj))
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8016.062  8100.824 -3997.031  7994.062     16398 
Random effects:
 Groups   Name        Std.Dev. Corr
 Subj     (Intercept) 0.13429      
 Subj.1   FHF         0.08828      
          FLF         0.08438  1.00
 Subj.2   Prel        0.08420      
          Punr        0.06106  1.00
 Residual             0.30559      
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)           F1           P1  
   -1.63965      0.01831      0.01867  


### Zerocorr LMM  -  default syntax (factors) -- does not return expected results!

In [23]:
R"""
m1 <- lmer((-1000/rt) ~ 1 + F + P + (1 | Subj) + (0 + F | Subj) + (0 + P | Subj), 
           dat, REML=FALSE, control=lmerControl(calc.derivs=FALSE))
"""

RObject{S4Sxp}
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: (-1000/rt) ~ 1 + F + P + (1 | Subj) + (0 + F | Subj) + (0 + P |  
    Subj)
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
 8016.062  8100.824 -3997.031  7994.062     16398 
Random effects:
 Groups   Name        Std.Dev. Corr
 Subj     (Intercept) 0.13429      
 Subj.1   FHF         0.08828      
          FLF         0.08438  1.00
 Subj.2   Prel        0.08420      
          Punr        0.06106  1.00
 Residual             0.30559      
Number of obs: 16409, groups:  Subj, 73
Fixed Effects:
(Intercept)           F1           P1  
   -1.63965      0.01831      0.01867  


The LRT favors the complex LMM, but not that  χ² < 2*(χ²-dof). 

## A replication of MRK17 LMM

### LMM with indicator variables 

Replication of final LMM in Masson and Kliegl (2013, Table 1) as well as reproduction of final lme4-based LMM in Masson et al. (2017, Figure 2).

In [24]:
m3form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        (1+q | Subj) + (0+lt | Subj) + (1 | Item) + (0 + p | Item) ;
mrk17_LMM = fit(LinearMixedModel, m3form, dat, contrasts=cntrsts);

VarCorr(mrk17_LMM)

Variance components:
            Column      Variance     Std.Dev.    Corr.
Item     (Intercept)  0.00320564078 0.056618378
         p            0.00006688990 0.008178625   .  
Subj     (Intercept)  0.03061639307 0.174975407
         q            0.00076244699 0.027612443 -0.42
         lt           0.00106206684 0.032589367   .     .  
Residual              0.08640808393 0.293952520


### LMM with factors

In [25]:
m3form_b = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
        (1+Q | Subj) + zerocorr(0+lT | Subj) + zerocorr(1 + P | Item) ;
mrk17_LMM_b = fit(LinearMixedModel, m3form_b, dat, contrasts=cntrsts);

VarCorr(mrk17_LMM_b)

Variance components:
            Column      Variance     Std.Dev.    Corr.
Item     (Intercept)  0.00320564078 0.056618378
         P: unr       0.00006688990 0.008178625   .  
Subj     (Intercept)  0.03061639307 0.174975407
         Q: deg       0.00076244699 0.027612443 -0.42
         lT: NW       0.00106206684 0.032589367   .     .  
Residual              0.08640808393 0.293952520


### Is the correlation parameter significant?

In [26]:
# remove single CP for nested LMMs
m4form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        zerocorr(1+Q + lT | Subj)+ zerocorr(1 + P | Item) ;
rdcdLMM = fit(LinearMixedModel, m4form, dat, contrasts=cntrsts);
VarCorr(rdcdLMM)

Variance components:
            Column      Variance     Std.Dev.    Corr.
Item     (Intercept)  0.00320752804 0.056635043
         P: unr       0.00006803353 0.008248244   .  
Subj     (Intercept)  0.03062344020 0.174995543
         Q: deg       0.00076344621 0.027630530   .  
         lT: NW       0.00106532575 0.032639328   .     .  
Residual              0.08640533704 0.293947847


In [27]:
MixedModels.likelihoodratiotest(rdcdLMM, mrk17_LMM)

Model Formulae
1: :(-1000 / rt) ~ 1 + f + p + q + lq + lt + f & p + f & q + p & q + f & lq + p & lq + q & lq + f & lt + p & lt + q & lt + lq & lt + f & p & q + f & p & lq + f & q & lq + p & q & lq + f & p & lt + f & q & lt + p & q & lt + f & lq & lt + p & lq & lt + q & lq & lt + f & p & q & lq + f & p & q & lt + f & p & lq & lt + f & q & lq & lt + p & q & lq & lt + f & p & q & lq & lt + MixedModels.ZeroCorr((1 + Q + lT | Subj)) + MixedModels.ZeroCorr((1 + P | Item))
2: :(-1000 / rt) ~ 1 + f + p + q + lq + lt + f & p + f & q + p & q + f & lq + p & lq + q & lq + f & lt + p & lt + q & lt + lq & lt + f & p & q + f & p & lq + f & q & lq + p & q & lq + f & p & lt + f & q & lt + p & q & lt + f & lq & lt + p & lq & lt + q & lq & lt + f & p & q & lq + f & p & q & lt + f & p & lq & lt + f & q & lq & lt + p & q & lq & lt + f & p & q & lq & lt + (1 + q | Subj) + (0 + lt | Subj) + (1 | Item) + (0 + p | Item)
─────────────────────────────────────────────────
     model-dof   deviance      χ²  χ²-dof

The correlation parameter was replicated (i.e., -.42 in MRK17) and significant. 

In [28]:
mods2 = [rdcdLMM, mrk17_LMM];
gof_summary = DataFrame(dof=dof.(mods2), deviance=deviance.(mods2),
              AIC = aic.(mods2), AICc = aicc.(mods2), BIC = bic.(mods2))

,dof,deviance,AIC,AICc,BIC
,Int64,Float64,Float64,Float64,Float64
1,38,7195.87,7271.87,7272.06,7564.69
2,39,7186.82,7264.82,7265.01,7565.33


Here `dof` or degrees of freedom is the total number of parameters estimated in the model and `deviance` is simply negative twice the log-likelihood at convergence, without a correction for a saturated model.  The AIC/BIC information criteria are on a scale of "smaller is better" and all would select `mrk17_LMM` as "best".

### Fixed effects

Finally, a look at the fixed effects. The four-factor interaction reported in Masson & Kliegl (2013) was not replicated.

In [29]:
show(mrk17_LMM)

Linear mixed model fit by maximum likelihood
 :(-1000 / rt) ~ 1 + f + p + q + lq + lt + f & p + f & q + p & q + f & lq + p & lq + q & lq + f & lt + p & lt + q & lt + lq & lt + f & p & q + f & p & lq + f & q & lq + p & q & lq + f & p & lt + f & q & lt + p & q & lt + f & lq & lt + p & lq & lt + q & lq & lt + f & p & q & lq + f & p & q & lt + f & p & lq & lt + f & q & lq & lt + p & q & lq & lt + f & p & q & lq & lt + (1 + q | Subj) + (0 + lt | Subj) + (1 | Item) + (0 + p | Item)
     logLik        -2 logLik          AIC             BIC       
 -3.59340856×10³  7.18681711×10³  7.26481711×10³  7.56533494×10³

Variance components:
            Column      Variance     Std.Dev.    Corr.
Item     (Intercept)  0.00320564078 0.056618378
         p            0.00006688990 0.008178625   .  
Subj     (Intercept)  0.03061639307 0.174975407
         q            0.00076244699 0.027612443 -0.42
         lt           0.00106206684 0.032589367   .     .  
Residual              0.08640808393 0.293952520


# Appendix 

## Weave the document in the REPL

```
julia> using Weave
julia> weave(scriptsdir("MRK17_spcfctn_slctn.jmd"), doctype="md2html")
```

## Switch to jupyter notebook from REPL

```
julia> using Weave, IJulia
julia> convert_doc(scriptsdir("MRK17_spcfctn_slctn.jmd"), projectdir("notebooks","MRK17_spcfctn_slctn.ipynb"))
julia> IJulia.notebook(dir=projectdir("notebooks"))
```

## Info

In [30]:
using InteractiveUtils
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i9-8950HK CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = "/Applications/Visual Studio Code.app/Contents/Resources/app/bin/code"
  JULIA_NUM_THREADS = 6
